In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv("/kaggle/input/heartbeat/mitbih_train.csv", header=None)
df_test = pd.read_csv("/kaggle/input/heartbeat/mitbih_test.csv", header=None)
df_train.head()

In [ ]:
plt.plot(df_train.iloc[0,:186])

In [ ]:
df_train[187].value_counts()

In [ ]:
c0 = df_train[df_train[187]==0].sample(1000)
c1 = df_train[df_train[187]==1].sample(1000)
c2 = df_train[df_train[187]==2].sample(1000)
c3 = df_train[df_train[187]==3]
c4 = df_train[df_train[187]==4].sample(1000)
df_train = pd.concat([c0, c1, c2, c3, c4])

In [ ]:
X_train = df_train.values[:, :-1]
y_train = df_train.values[:, -1].astype(int)

X_test = df_test.values[:, :-1]
y_test = df_test.values[:, -1].astype(int)

In [ ]:
X_train = np.resize(X_train, (X_train.shape[0], 187, 1))
X_test = np.resize(X_test, (X_test.shape[0], 187, 1))
print(X_train.shape)
print(X_test.shape)

In [ ]:
from tensorflow.keras import utils
y_train = utils.to_categorical(y_train, 5)
print(y_train.shape)

In [ ]:
n_obs, feature, depth = X_train.shape
num_classes= 5
print(feature, depth)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout

In [ ]:
model = Sequential()
model.add(Conv1D(32, 3, padding='same', input_shape=(187, 1), activation='relu'))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(Conv1D(128, 3, padding='same', activation='relu'))

model.add(Flatten())
model.add(Dense(5, activation='softmax'))

In [ ]:
from  tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
rlp = lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 2,verbose = 1,factor = 0.8, min_lr = 1e-6)
es = EarlyStopping(verbose=1, patience=2)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['AUC'])

In [ ]:
his = model.fit(X_train, y_train, epochs=10000, batch_size=100,validation_split=0.2, callbacks=[lrd,es])

In [ ]:
p = model.predict(X_test)
y_pred = []
print(p.shape)

In [ ]:
err = 0
for i in np.arange(p.shape[0]):
    y_pred.append(p[i].argmax())
    if(p[i].argmax() != int(y_test[i])):
        err += 1
print("正確率 :", 1 - err/p.shape[0])

In [ ]:
from sklearn.metrics import confusion_matrix

C=confusion_matrix(y_test, y_pred)
print(C)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))